<a href="https://colab.research.google.com/github/MaksimReznicenko/Maksim_Reznichenko/blob/main/%D0%A0%D0%956_2_%22LangChain__LangModel%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LangChain**


In [ ]:
#Установим необходимые библиотеки:
!pip install -qU langchain
!pip install -qU langchain-openai
!pip install -qU langchain-anthropic
!pip install -qU langchain-community
!pip install -qU wikipedia

# или !pip install -qU langchain langchain-openai wikipedia

https://platform.openai.com/account/api-keys

https://smith.langchain.com/

https://www.youtube.com/watch?v=VOwI5RsPyzc - Как использовать прокси в браузере

In [ ]:
from langchain_openai import ChatOpenAI
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"]="добавляем свой ключ"
os.environ["OPENAI_API_KEY"]="добавляем свой ключ"

In [ ]:
model = ChatOpenAI(
	model="gpt-4o-mini",
	temperature=0.7,
	max_tokens=1000
	)

In [ ]:
os.environ["ANTHROPIC_API_KEY"] ="123"
from langchain_anthropic import ChatAnthropic
model_claude = ChatAnthropic(model="claude-3-sonnet-20240229")

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
SystemMessage(content="Translate the following from English into Italian"),
HumanMessage(content="hi!"),

]

model.invoke(messages)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
result = model.invoke(messages)
parser.invoke(result)

In [ ]:
chain = model | parser
chain.invoke(messages)

Проверим LangSmith!

In [ ]:
#получим стоимость запроса
from langchain_community.callbacks.manager import get_openai_callback

with get_openai_callback() as cb:
    result = chain.invoke(messages)
    print(cb)

In [ ]:
#проверим другую модель
model4 = ChatOpenAI(model="gpt-4o")
chain4 = model4 | parser

with get_openai_callback() as cb:
    result = chain4.invoke(messages)
    print(cb)

In [ ]:
print(f"Total Cost (USD): {cb.total_cost}$")

In [ ]:
#Используем шаблон промпта
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [ ]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

In [ ]:
#доступ к сообщениям
result.to_messages()#[0]

In [ ]:
result.to_messages()[0]

In [ ]:
#добавляем шаблон
chain = prompt_template | model | parser

In [ ]:
chain.invoke({"language": "italian", "text": "hi"})

In [ ]:
#Пример использования
system_template = "Ты - полезный бот массовик затейник. Ты знаешь много способов провести интересно время. Тебя зовут {name}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
    ("user", "Меня зовут Михаил. Я люблю карточные игры, особенно покер. Хочу научиться играть в преферанс. \
        Не люблю гулять на свежем воздухе"),

        ("user", "{user_input}")]

)

result = prompt_template.invoke({"name": "Бодрый друг", "user_input": "На улице солнечно. Чем посоветуешь заняться?"})

chain = prompt_template | model | parser
data = {"name": "Бодрый друг", "user_input": "На улице солнечно. Чем посоветуешь заняться?"}
chain.invoke(data)

In [ ]:
#ответ в режиме стрима
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    )
for chunk in chat.stream("Напиши тезисы про важность промпт-инжиниринга"):
    print(chunk.content, end="", flush=True)

 LangChain. Взаимодействие с языковыми моделями: Часть 2

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

# Определение Pydantic модели для структурированного вывода
class Person(BaseModel):
    name: str = Field(description="Полное имя человека")
    age: int = Field(description="Возраст человека")
    hobbies: List[str] = Field(description="Список хобби")

# Создание парсера
parser = PydanticOutputParser(pydantic_object=Person)
parser.get_format_instructions().encode('utf-8').decode('unicode-escape')

In [ ]:
# Создание шаблона промпта
prompt = PromptTemplate(
    template="Extract structured information from the following text:\n{text}\n{format_instructions}",
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Создание цепочки
chain = prompt | model | parser

text = """
Иван Иванов - 32-летний инженер-программист. Он любит играть на гитаре,
ходить в походы по выходным и экспериментировать с новыми кулинарными рецептами.
"""

result = chain.invoke({"text": text})

print(result)

In [ ]:
print(f"Name: {result.name}")
print(f"Age: {result.age}")
print(f"Hobbies: {', '.join(result.hobbies)}")

In [ ]:
from langchain.globals import set_debug
set_debug(True)
result = chain.invoke({"text": text})
set_debug(True)

In [ ]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os

# Определите вашу модель
llm = OpenAI(temperature=0, max_tokens=3000)

# Создайте шаблон для суммаризации текста
prompt_template = """
Пожалуйста, подведите итог следующему тексту:
{text}
"""

# Инициализация цепочки с использованием LLM
prompt = PromptTemplate(input_variables=["text"], template=prompt_template)
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Текст для суммаризации
text = """
Команда ИИ-агентов создает гаджеты будущего без электроники!

Ученые из Калифорнийского университета в Беркли разработали систему автономных ИИ-агентов,
способную проектировать устройства на основе "жидкостных вычислений". Эта технология позволяет
создавать умные гаджеты, работающие без электроники, используя лишь пневматические схемы.
Вместо микросхем - воздушные клапаны, вместо проводов - трубки. И всё это проектирует искусственный интеллект.

Система использует мультиагентный подход: один ИИ-агент генерирует идеи устройств,
другой прорабатывает детали конструкции, третий проверяет их на реализуемость.
В результате получаются проекты вроде йога-мата, который сам подсказывает правильные позы,
надуваясь в нужных местах, или кресла, корректирующего вашу осанку без единого датчика.
ИИ не просто фантазирует - он учитывает все ограничения технологии и выдает только те проекты,
которые можно воплотить в реальность. При этом система способна за считанные минуты сгенерировать
десятки вариантов применения технологии в различных сферах - от медицины до развлечений.

Эта технология может изменить целые отрасли. Производители бытовой техники и мебели получат
возможность создавать продукты с новым уровнем функциональности. Медицинская промышленность
сможет разрабатывать более доступные и надёжные устройства для реабилитации. А дизайнеры получат
мощный инструмент для воплощения самых смелых идей.
"""

# Запустите цепочку
summary = llm_chain.run(text=text)
print(summary)

In [ ]:
#используем примеры few shot
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
        Are follow up questions needed here: Yes.
        Follow up: How old was Muhammad Ali when he died?
        Intermediate answer: Muhammad Ali was 74 years old when he died.
        Follow up: How old was Alan Turing when he died?
        Intermediate answer: Alan Turing was 41 years old when he died.
        So the final answer is: Muhammad Ali
        """,
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
        Are follow up questions needed here: Yes.
        Follow up: Who was the founder of craigslist?
        Intermediate answer: Craigslist was founded by Craig Newmark.
        Follow up: When was Craig Newmark born?
        Intermediate answer: Craig Newmark was born on December 6, 1952.
        So the final answer is: December 6, 1952
        """,
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
        Are follow up questions needed here: Yes.
        Follow up: Who was the mother of George Washington?
        Intermediate answer: The mother of George Washington was Mary Ball Washington.
        Follow up: Who was the father of Mary Ball Washington?
        Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
        So the final answer is: Joseph Ball
      """,
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
        Are follow up questions needed here: Yes.
        Follow up: Who is the director of Jaws?
        Intermediate Answer: The director of Jaws is Steven Spielberg.
        Follow up: Where is Steven Spielberg from?
        Intermediate Answer: The United States.
        Follow up: Who is the director of Casino Royale?
        Intermediate Answer: The director of Casino Royale is Martin Campbell.
        Follow up: Where is Martin Campbell from?
        Intermediate Answer: New Zealand.
        So the final answer is: No
        """,
    },
]

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

print(example_prompt.format(**examples[2]))

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

In [ ]:
chain = prompt | model

chain.invoke({"input": "Как звали супруг первого президента Российской Федерации?"})

**Function calling**

In [ ]:
!pip install langchain-community
!pip install pyowm

In [ ]:
# Импортируем необходимые библиотеки для работы с инструментами и API
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import os

# Устанавливаем ключи API для Tavily и OpenWeatherMap
os.environ["TAVILY_API_KEY"] = 'добавить API'
os.environ["OPENWEATHERMAP_API_KEY"] = 'добавить API'

In [ ]:
# Это шаблон, который будет использоваться для построения диалога между системой и пользователем
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),  # Настроим систему как помощника
        ("placeholder", "{chat_history}"),        # Место для истории чата
        ("human", "{input}"),                      # Место для запроса от пользователя
        ("placeholder", "{agent_scratchpad}"),     # Место для промежуточных вычислений агента
    ]
)

In [ ]:
# Создаём инструмент для получения информации о погоде через OpenWeatherMap API
weather = OpenWeatherMapAPIWrapper()
weather_tool = Tool(
    name="Weather",
    func=weather.run,  # Важно: эта функция будет использоваться для получения прогноза погоды
    description="Useful for getting weather information"
)

# Создаём инструмент для поиска через Tavily API (поиск по месту)
tools = [TavilySearchResults(max_results=7), weather_tool]

In [ ]:
# Используем модель OpenAI для создания агента
model = ChatOpenAI(temperature=0.7)  # Температура 0.7 задает умеренную случайность ответов

# Создаём агент, который использует инструменты (Tavily для поиска и OpenWeatherMap для погоды)
agent = create_openai_functions_agent(model, tools, prompt)

# Создаём исполнитель агента
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# Пример 1: Запрос прогноза погоды
print("Пример 1: Запрос прогноза погоды")
response = agent_executor.invoke({"input": "будет ли в Москве дождь?"})
print(response)  # Выводим ответ агента на запрос

In [ ]:
# Пример 2: Запрос о месте через Tavily
print("\nПример 2: Запрос о месте для путешествия")
response = agent_executor.invoke({"input": "Какие интересные места для отдыха в Италии?"})
print(response)  # Выводим ответ агента на запрос

In [ ]:
# Пример 3: Философский вопрос, на который агент отвечает без использования API
print("\nПример 3: Философский вопрос")
response = agent_executor.invoke({"input": "Есть ли в жизни счастье?"})
print(response)  # Выводим ответ агента на философский вопрос

**ReAct Аgent**


Action: выбор инструмента

Action Input: передача данных в инструмент

Observation: наблюдение за ответом

Thought: вывод

In [ ]:
# ReAct agent
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [ ]:
from pprint import pprint as pp
a = 'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

pp(a)

In [ ]:
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "будет ли в Москве дождь?"})